In [ ]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import time

In [ ]:
db_news = sqlite3.connect('./raws.db')
c = db_news.cursor()
# 이미 table이 있는 경우 주석처리
c.execute("CREATE TABLE newspapers (id INTEGER PRIMARY KEY AUTOINCREMENT, title TEXT, press TEXT, category TEXT, date TEXT, name TEXT, text TEXT)")

In [ ]:
browser = webdriver.Chrome('C:\chromedriver.exe')
browser.get('https://www.bigkinds.or.kr/v2/news/index.do')

# 언론사 분류
str = '삭제경향신문삭제국민일보삭제내일신문삭제동아일보삭제문화일보삭제서울신문삭제세계일보삭제조선일보삭제중앙일보삭제한겨레삭제한국일보삭제매일경제삭제머니투데이삭제서울경제삭제아시아경제삭제아주경제삭제파이낸셜뉴스삭제한국경제삭제헤럴드경제삭제강원도민일보삭제강원일보삭제경기일보삭제경남도민일보삭제경남신문삭제경상일보삭제경인일보삭제광주매일신문삭제광주일보삭제국제신문삭제대구일보삭제대전일보삭제매일신문삭제무등일보삭제부산일보삭제영남일보삭제울산매일삭제전남일보삭제전북도민일보삭제전북일보삭제제민일보삭제중도일보삭제중부매일삭제중부일보삭제충북일보삭제충청일보삭제충청투데이삭제한라일보삭제KBS삭제MBC삭제OBS삭제SBS삭제YTN삭제디지털타임스삭제전자신문삭제'
press_str = str.split('삭제')
pres_list = press_str[1:-1]
print(pres_list)

# 기간 설정 ['시작날짜', '마지막날짜']
date = ['2021-01-01', '2021-07-31']

# 언론사 설정 press_list 中 1개 선택
press = pres_list[0]
print(press)

browser.find_element_by_xpath("//div[@class='tab-btn-inner tab1']").click()

start_date = browser.find_element_by_xpath("//input[@id='search-begin-date']")
start_date.click()
start_date.send_keys(Keys.CONTROL + "a")
start_date.send_keys(date[0])

end_date = browser.find_element_by_xpath("//input[@id='search-end-date']")
end_date.click()
end_date.send_keys(Keys.CONTROL + "a")
end_date.send_keys(date[1])

browser.find_element_by_xpath("//div[@class='tab-btn-inner tab2']").click()

press_ = browser.find_element_by_xpath("//label[contains(text() ,'" + press + "')]")
press_.click()

time.sleep(1)
browser.find_element_by_xpath("//button[@class='btn btn-search news-search-btn news-report-search-btn']").click()


In [ ]:
# 반복 시행 해야할 부분

In [ ]:
for p in range(0, 10):
    WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, "//select[@id='select2']/option[4]")))
    browser.find_element_by_xpath("//select[@id='select2']/option[4]").click()

    WebDriverWait(browser, 2).until(EC.presence_of_element_located((By.CLASS_NAME, "news-inner")))
    time.sleep(1)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')

    wholes = soup.select('div.news-inner')
    len(wholes)
    
    contents = [] # [제목, 언론사, 분류, 날짜, 기자이름, 기사내용]
    for whole in wholes:
        title = whole.select('span.title-elipsis')
        press =  whole.select('div.info > div > a')
        category = whole.select('div.info > div > span.bullet-keyword')
        date_name = whole.select('div.info > p')
        contents.append([title[0].text.strip(), press[0].text.strip(), category[0].text.strip(), date_name[0].text.strip(), date_name[1].text.strip()])
    
#     WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.title-elipsis")))
    texts = []
    for i in range(0, 100):
        time.sleep(1)
        browser.find_elements_by_css_selector('span.title-elipsis')[i].click()
        WebDriverWait(browser, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.news-view-body")))
        time.sleep(1)
        text = browser.find_elements_by_css_selector('div.news-view-body')[0].text
        texts.append(text.replace('\n', ''))
        WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.XPATH, "//div[@id='news-detail-modal']/div/div/button")))
        time.sleep(1)
        browser.find_element_by_xpath("//div[@id='news-detail-modal']/div/div/button").click()

    for i in range(0,len(texts)):
        contents[i].append(texts[i])

    for content in contents:
        title = content[0]
        press = content[1]
        category = content[2]
        date = content[3]
        name = content[4]
        text = content[5]
        c.execute("INSERT INTO newspapers (title, press, category, date, name, text) VALUES(?,?,?,?,?,?) ",(title, press, category, date, name, text))
    time.sleep(1)
    db_news.commit()

    next_p = browser.find_element_by_xpath("//a[@class='page-next page-link']")
    browser.execute_script("arguments[0].click();", next_p)